In [ ]:
from pathlib import Path

from joblib import dump
from tira.rest_api_client import Client
import pandas as pd
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from sklearn.feature_extraction.text import CountVectorizer
import re

# Load the data
tira = Client()
text = tira.pd.inputs(
    "nlpbuw-fsu-sose-24", "language-identification-train-20240429-training"
)
#text = text.set_index("id")
labels = tira.pd.truths(
    "nlpbuw-fsu-sose-24", "language-identification-train-20240429-training"
)

df = text.join(labels.set_index("id"))

# TODO : classify all non-latin-languages, save all remaining latin-languages as "text_val_latin_only"
text_val_latin_only = text

remove_punctuation = str.maketrans('', '', r"-()\"#/@;:<>{}-=~|.?,")

def PunctFreeLower(texts):
    cleaned = []
    for text in texts:
        cleaned.append((re.sub(r"[0-9]+", "", (text.translate(remove_punctuation)))).lower())
    return cleaned

text_val_cleaned = PunctFreeLower(text_val_latin_only["text"])

vec2 = CountVectorizer(analyzer='char', ngram_range=(1, 3))
text_val_vec2 = vec2.transform(text_val_cleaned)
   

# Load the model and make predictions
model = open(Path().resolve() / "model.joblib")
predictions = model.predict(text_val_vec2)
df["language"] = predictions
df = df[["id", "language"]]

# Save the predictions
df.to_json(
    Path().resolve() / "predictions.jsonl", orient="records", lines=True
)